In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

df = pd.read_csv('../input/basketball-players-stats-per-season-49-leagues/players_stats_by_season_full_details.csv')
df = df[df['League'].eq('NBA')]

# Метод K-средних и кластер-анализ

1. Выберете не менее трех количественных признаков, объясните выбор и примените метод K-средних для K=5 и 9. В обоих случаях сделайте порядка 10 случайных инициализаций и выберите то, которое доставляет минимум критерию метода;
2. Проинтерпретируйте оба разбиения с помощью помощью признаков таблицы данных путем сравнения внутри-кластерных средних с общим средним, т.е. вычисления относительных отклонений внутри-кластерных средних от общего среднего. Объясните, какое разбиение лучше с точки зрения интерпретации;

Выбрал: рост (ключевой физический показатель для этой игры), и ряд признаков, характеризующих игру в разных амплуа (защита, атака и тд) (например, большое значения `BLK` и `REB` говорит, что перед нами скорее защитник; или значение выше среднего по `3PM` подсказывает, что перед нами хороший атакующий)

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

_, ax = plt.subplots(2, figsize=(25, 15))

cluster_cols = ['PTS', 'STL', 'REB', 'PF', 'TOV', 'AST', 'BLK', '3PM', 'height_cm']
grand_mean_df = pd.DataFrame(df[cluster_cols].mean().values.reshape(1, -1), index=['Grand mean'], columns=cluster_cols)

for k in [5, 9]:
    kmeans = KMeans(n_clusters=k, n_init=10, random_state=42)
    sc = StandardScaler()
    kmeans.fit(sc.fit_transform(df[cluster_cols]))
    kmean_name = f'KMeans_{k}_labels'
    df[kmean_name] = kmeans.labels_

    i = k // 9
    graph_df = pd.concat((df[cluster_cols + [kmean_name]].groupby(kmean_name).mean(), grand_mean_df))
    graph_df.plot(kind='bar', ax=ax[i])
    ax[i].grid(axis='y')

    ax[i].tick_params(axis="x", labelsize=15, rotation=0)
    ax[i].tick_params(axis="y", labelsize=15)
#     ax[i].legend(prop={'size': 15})

    ax[i].set_ylabel('cluster center values', fontsize=20)
    ax[i].set_xlabel('cluster', fontsize=20)
    ax[i].set_title(f'KMeans {k} clusters', fontsize=25)



In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

_, ax = plt.subplots(2, figsize=(25, 15))

cluster_cols = ['PTS', 'STL', 'REB', 'PF', 'TOV', 'AST', 'BLK', '3PM', 'height_cm']

for k in [5, 9]:
    kmeans = KMeans(n_clusters=k, n_init=10, random_state=42)
    kmeans.fit(StandardScaler().fit_transform(df[cluster_cols]))

    i = k // 9
    pd.DataFrame(kmeans.cluster_centers_, columns=cluster_cols).plot(kind='bar', ax=ax[i])
    ax[i].grid(axis='y')

    ax[i].tick_params(axis="x", labelsize=15, rotation=0)
    ax[i].tick_params(axis="y", labelsize=15)
#     ax[i].legend(prop={'size': 15})

    ax[i].set_ylabel('cluster center values', fontsize=20)
    ax[i].set_xlabel('cluster', fontsize=20)
    ax[i].set_title(f'KMeans {k} clusters', fontsize=25)



Так как я стандартизовал данные для корректной работы K-means, то общее среднее каждого признака равно нулю и можно его никак не отображать на графиках. 

Проинтерпретируем кластеры первого разбиения (названия брал [отсюда](https://ru.wikipedia.org/wiki/%D0%9F%D0%BE%D0%B7%D0%B8%D1%86%D0%B8%D0%B8_%D0%B2_%D0%B1%D0%B0%D1%81%D0%BA%D0%B5%D1%82%D0%B1%D0%BE%D0%BB%D0%B5)).
0. Высокие показатели по защите и средние по атаке. Интерпретирую этот кластер, как кластер защитников.
1. Скорее всего, это кластер новичков. У них средний рост (так как в этом кластере новые игроки разных амплуа) и все игровый показатели сильно ниже среднего. Полагаю, что они пока проводят недостаточно времени на поле, чтобы показывать какие-то результаты 
2. Рост ниже среднего, высокие показатели по атаки и в особенности по голевым передачам (`AST`). Все это наводит на мысль, что это кластер центровых.
3. Высокие показатели по трехочковым и перехватам и низкий рост. Кластер похож на кластер центровых или разыгрываюищих.
4. Тут все достаточно очевидно: высокий рост, высокие показатели по подборам (`REB`) и блокам (`BLK`). Понятно, что это кластер тяжелых форвардов.


Теперь кратко опишем кластеры во втором разбиении:
0. Похоже на средних игроков, ничем не выдающихся.
1. Новички с высоким ростом (мотивацию объяснял в предыдущих комментариях)
2. Тяжелый форвард (высокий рост, блоки, подборы)
3. Атакующий защитник (хорошие показатели по трехочковым – свойство атакующего защитника)
4. Разгрывающие (низкий рост и хорошие показатели по голевым передачами и перехватам)
5. Защитники (выдаются только показатели, связанные с защитой)
6. Новички с низким ростом
7. Разыгрывающие защитники (рост ниже среднего; высокие показатели перехватов, очков, голевых передач)
8. Центровой (высокий рост; хорошие показатели по атаке, в особенности подборами)

# Бутстрэп

Для одного из полученных в предыдущем задании разбиений:
1. Найдите 95% доверительный интервал для среднего значения какого-либо признака на всем множестве объектов, используя бутсрэп;
2. Сравните средние по какому-либо признаку в двух кластерах, используя бутсрэп;
3. Для одного из кластеров сравните среднее на всем множестве для какого-либо признака с его средним внутри кластера, используя бутсрэп;


* Откомментируйте результаты;
* Примечание: каждое применение бутстрэпа должно быть обоими методами, с опорой и без;

__Рассмотрим рост__, то есть `height_cm`

Для начала взглянем на его гистограмму

In [ ]:
import seaborn as sns

_, ax = plt.subplots(figsize=(15,7))

sns.distplot(df['height_cm'], bins=20, ax=ax)

ax.tick_params(axis="x", labelsize=15)
ax.tick_params(axis="y", labelsize=15)

ax.set_xlabel('height_cm', fontsize=20)
ax.set_ylabel('density', fontsize=20)
ax.set_title('height_cm distribution', fontsize=30)

# ax.set_axisbelow(False)
ax.grid(axis='y')

Можно заметить скачки вблизи круглых чисел (190, 200, 210). Наверное, потому что многие давали округленные данные.

Теперь рассмотрим гистограмму для бутстрэпа

In [ ]:
features = df['height_cm'].values
n_bootstrap = 5000
bs = np.mean(np.random.choice(features, size=(len(features), n_bootstrap)), axis=0)

_, ax = plt.subplots(figsize=(15,7))

sns.distplot(bs, bins=20, ax=ax)

ax.tick_params(axis="x", labelsize=15)
ax.tick_params(axis="y", labelsize=15)

ax.set_xlabel('height_cm', fontsize=20)
ax.set_ylabel('density', fontsize=20)
ax.set_title('resampled height_cm distribution', fontsize=30)

# ax.set_axisbelow(False)
ax.grid(axis='y')

После ресэмплинга гистограмма сгладилась (так как ЦПТ).

Теперь найдем доверительный интервал для всей выборки

In [ ]:
# stolen from assist's notebook
def bootstrap(x, n_bootstrap=n_bootstrap):
    
    # pivotal
    mean_p = np.mean(x)
    std_p = np.std(x)
    lbord_p = mean_p - 1.96 * std_p
    rbord_p = mean_p + 1.96 * std_p
    confint_p = np.array([lbord_p, rbord_p])
    
    # non-pivotal
    x_sorted = np.sort(x)
    lbord_np = x_sorted[int(n_bootstrap * 0.025) + 1]
    rbord_np = x_sorted[int(n_bootstrap * 0.975)]
    confint_np = np.array([lbord_np, rbord_np])
    
    return confint_p, confint_np


rm_confint_p, rm_confint_np = bootstrap(bs)
print(f'Feature mean: {np.mean(features)}')
print(f'Confidence interval (pivotal): [{rm_confint_p[0]}; {rm_confint_p[1]}]')
print(f'Confidence interval (non-pivotal): [{rm_confint_np[0]}; {rm_confint_np[1]}]')


Второй интервал получился уже. (думаю, что тут все в пределах погрешности и это ни о чем не говорит)

Теперь сравним бутстрэпы в двух кластерах (1 и 3 для первой кластеризации)

In [ ]:
_, ax = plt.subplots(1, 2, figsize=(15,7))

for k in [1, 3]:
    features = df.loc[df['KMeans_5_labels'].eq(k),'height_cm'].values
    n_bootstrap, i = 5000, k // 3
    bs = np.mean(np.random.choice(features, size=(len(features), n_bootstrap)), axis=0)
    sns.distplot(bs, bins=20, ax=ax[i])

    ax[i].tick_params(axis='x', labelsize=15)
    ax[i].tick_params(axis='y', labelsize=15)

    ax[i].set_xlabel('height_cm', fontsize=20)
    ax[i].set_ylabel('density', fontsize=20)
    ax[i].set_title(f'cluster {k}', fontsize=30)

    # ax.set_axisbelow(False)
    ax[i].grid(axis='y')

Видно, что средние обоих кластерах четко различаются. В левом люди в среднем выше. Также скажем, что среднее в общем сильно отличается от среднего внутри рассмотренных кластеров.

# Таблицы сопряжённости

In [ ]:
df[cluster_cols]

In [ ]:
features = []
features.append(df["STL"])
features.append(df["AST"])
features.append(df["PF"])

In [ ]:
plt.hist(features[0], 60)
plt.yscale("log")
plt.axvline(15, c='r')
plt.axvline(100, c='r')

Я взял логарифмическую шкалу, так как выбрать класс оказывается сложно в линейной шкале

In [ ]:
plt.hist(features[1], 60)
plt.yscale("log")
plt.axvline(30, c='r')
plt.axvline(420, c='r')
plt.axvline(760, c='r')

Аналогично и в данном распределении: классы в линейной шкале не так заметны, как здесь

In [ ]:
plt.hist(features[2], 60)
plt.axvline(75, c='r')
plt.axvline(250, c='r')

Третий признак (число фолов), напротив, имеет ярко выделенные классы

In [ ]:
feature_classes = []
feature_classes.append(pd.cut(features[0], [0, 15, 100, 300], labels=['S1', 'S2', 'S3']))
feature_classes.append(pd.cut(features[1], [0, 30, 420, 760, 1000], labels=['A1', 'A2', 'A3', 'A4']))
feature_classes.append(pd.cut(features[2], [0, 75, 250, 400], labels=['P1', 'P2', 'P3']))

Рассмотрим зависимость совместной подачи (после которой следующий игрок забивает очко) от числа украденных мячей

In [ ]:
cond = feature_classes[0]
targ = feature_classes[1]
ct = pd.crosstab(targ, cond, margins=True)
ct

Стандартная таблица сопряжённости нам ни о чём не говорит, поэтому возьмём частоту каждого события

In [ ]:
rf = pd.crosstab(targ, cond, margins=True, normalize=True)
rf.round(2)

Теперь результаты становятся интересней. Мы можем отметить, что в категории с малым числом украденных мячей мало совместных подач, а наибольшее число игроков совершает среднее число краж, и среднее число совместных подач. Но, на самом деле, эти числа зависели от нашего разбиения, поэтому построим таблицу условных вероятностей, чтобы иметь представление о реальной картине

In [ ]:
cf = np.divide(ct, ct[-1:])
cf.round(2)

В результате мы можем отметить вышеперечисленные тренды: в случае малого и среднего числа краж лидирует, соответственно, малое и среднее число совместных подач. Однако, группа с большим числом краж, выбивается из этих расчётов, показывая преобладание в среднем числе совместных подач. Из этого можно сделать вывод о том, что игроки, находящиеся в защите и совершающие наибольшее число краж, редко передают их атакующим, которые сразу после них забивают очко.

In [ ]:
quetlet = rf / (rf["All"].values.reshape(-1, 1) @ rf.loc["All", :].values.reshape(1, -1)) - 1
quetlet

Матрица Кетле показывает нам, что в третьей группе наблюдается низкое отклонение от среднего (порядка 7-8%), а это значит, что игроки, отбирающие больше всего мячей, почти не связаны с игроками, совершающими совместную подачу. Тем не менее, первые два выделенных тренда заметны и здесь, с преобладанием в 171.5% и 41% соответственно, а значит данные совместные величины относятся к атакующим форвардам и разыгрывающим защитникам.

Из анализа, можем сделать вывод о том, что знание о принадлежности к первой или второй группе, даст нам высокую вероятность предсказать нужный класс. Убедимся в этом

In [ ]:
medi = rf * quetlet
round(np.sum(medi.values), 3)

In [ ]:
meanVals = rf["All"].values.reshape(-1, 1) @ rf.loc["All", :].values.reshape(1, -1)
xi2 = (rf - meanVals)**2 / meanVals
round(np.sum(xi2.values), 3)

Коэффициент хи-квадрат не отличается от среднего коэффициента Кетле

Таким образом, знание принадлежности игрока к классу по его числу украденных мячей, повышает вероятность предсказания его класса по числу совместных подач, на 81.3%

### Сделаем аналогичный разбор зависимости класса по украденным мячам от класса по числу фолов

In [ ]:
cond = feature_classes[0]
targ = feature_classes[2]
ct = pd.crosstab(targ, cond, margins=True)
rf = pd.crosstab(targ, cond, margins=True, normalize=True)
rf.round(2)

Аналогично, для получения подробной информации о зависимости, перейдём к условным вероятностям

In [ ]:
cf = np.divide(ct, ct[-1:])
cf.round(2)

Теперь мы видим, что каждый класс по отобранным мячам, очень сильно уходит в конкретный класс по фолам: в среднем, чем больше отобранных мячей, тем больше фолов. Отсюда мы можем сделать вывод о том, что большинство фолов совершаются при краже мяча. Кроме этого, мы можем заметить, что малый по количеству третий класс практически не отличается по распределению условных вероятностей от второго, а это значит, что здесь эти классы можно объединить 

In [ ]:
quetlet = rf / (rf["All"].values.reshape(-1, 1) @ rf.loc["All", :].values.reshape(1, -1)) - 1
quetlet

По таблице коэффициентов Кетле мы можем сделать вывод о том, что данные признаки оказываются очень сильно связаны статистически: знание о классе по украденным мячам очень сильно меняет представление о распределении классов по числу фолов - на десятки процентов от среднего. И это наблюдается в каждом классе.

In [ ]:
medi = rf * quetlet
round(np.sum(medi.values), 3)

Итак, мы видим, что средний индекс Кетле равен 0.699, а значит знание о классе по украденным мячам повышает шанс правильно предположить класс по числу фолов на 70%

In [ ]:
meanVals = rf["All"].values.reshape(-1, 1) @ rf.loc["All", :].values.reshape(1, -1)
xi2 = (rf - meanVals)**2 / meanVals
round(np.sum(xi2.values), 3)

Коэффициент хи-квадрат не отличается от среднего коэффициента Кетле

Сравнив значения индекса Кетле на обеих таблицах, мы можем сделать вывод о том, что знание о классе по украденным мячам, сильно повышают предсказания классов по совместным подачам и по совершённым фолам. Связь оказалась больше в первой таблице, согласно индексу Кетле, однако это возникло скорее всего по той причине, что в представленных данных не было чётко видимой границы разделения на несколько классов, поэтому пришлось сделать предположение. Были совершены также сравнения по слегка изменённым границам, которые в ту или иную сторону меняли индекс. В целом, стоит сказать, что зависимость получилась очень высокой.

Подсчёт коэффициентов хи-квадрат, дал вывод о том, что данный коэффициент в точности равен среднему коэффициенту Кетле, что и было доказано на лекциях. Отличия начинаются примерно в пятом знаке после запятой, что можно считать допустимой погрешностью

# МГК(PCA)/SVD

* Выберите в Ваших данных 3-6 признаков, более или менее относящихся к одному и тому же аспекту данных; откомментируйте.
* Визуализируйте Ваши данные дважды, один раз с использованием стандартизации размахом, второй – путем z-scoring. Выберите какую-либо группу объектов и выделите ее на визуализациях цветом или формой «точек».
* Примените традиционный метод МГК для визуализации (для какого-либо способа нормализации) и убедитесь, что получено то же самое, что и выше. Если нет – объясните, в чем дело.
* Постарайтесь проинтерпретировать полученные компоненты


In [ ]:
#columns = ["height_cm", "REB", "PTS", "BLK"]
columns = ["height_cm", "REB", "PTS", "BLK", "AST"]
data = df[df["Season"] == "2018 - 2019"][columns]
# data.iloc[:, 2] = data.iloc[:, 2] / data.iloc[:, 3]
# data.drop(columns=["MIN"])

Возьмем данные за 2018-2019
**Столбцы:**
* height_cm рост
* REB подборы
* PTS очки
* BLK блоки

In [ ]:
Xz = (data - data.mean(axis=0)) / data.std(axis=0) # Z-scoring
Xr = (data - data.mean(axis=0)) / (data.max(axis=0) - data.min(axis=0)) # range normalization

Отскалировали данные двумя способами

In [ ]:
def visualize_scaled_data(scaled_data, clusters=True, debug=True):
    uz, sz, vhz = np.linalg.svd(scaled_data.to_numpy())
    uz0 = uz[:, 0]
    vhz0 = vhz[0]
    uz1 = uz[:, 1]
    
    data_scatter = np.sum((scaled_data**2).values)
    contributions = sz**2 / data_scatter
    if debug:
        print("Singular value contributions:", contributions.round(2))
    
    plt.scatter(uz0, uz1)
    if clusters:
        plt.scatter(uz0[scaled_data.iloc[:, 0].values > 0], uz1[scaled_data.iloc[:, 0].values > 0], label="SVD")
    #plt.title('')

In [ ]:
visualize_scaled_data(Xz)

In [ ]:
**Посмотрим**: Разбили людей по росту. Видно что множества достаточно не пересекаются, то есть есть некоторое разбиение, что люди с высоким ростом, и с низким ростом в среднем имеют разные игровые показатели.

In [ ]:
visualize_scaled_data(Xr)

**Посмотрим:** Визуализация с использованием стандартизации размахом дала почти такое же представление как и в случае с z-scoring.

In [ ]:
def visualize_scaled_data_PCA_and_SVG(scaled_data):
    visualize_scaled_data(scaled_data, clusters=False, debug=False)
    
    N = scaled_data.shape[0]
    cov = np.dot(scaled_data.T, scaled_data) / N
    eigw, eigv = np.linalg.eig(cov)
    pca0 = np.dot(scaled_data, - eigv[:, 0]) / np.sqrt(N * eigw[0])
    cov = cov - eigw[0] * eigv[:, 0][:, None] * eigv[:, 0]
    eigw, eigv = np.linalg.eig(cov)
    pca1 = np.dot(scaled_data, eigv[:, 0]) / np.sqrt(N * eigw[0])
    
    plt.scatter(pca0, pca1, marker='.', label='covariance')
    plt.legend()
    plt.show()

In [ ]:
visualize_scaled_data_PCA_and_SVG(Xz)

**Вывод:** Визуализация методом PCA дает такой же вид, как визуализация методом SVD

# Домашнее задание 6

Возьмем очки (`PTS`) и потери (`TOV`)

In [ ]:
dff = df[df['Season'] == '2018 - 2019']
dff = dff[dff['TOV'] > 0]
plt.scatter(dff['PTS'],dff['TOV'])
plt.xlabel('PTS')
plt.ylabel('TOV')


In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression().fit(dff[['PTS']], dff[['TOV']])
plt.scatter(dff['PTS'],dff['TOV'])
plt.plot(dff['PTS'], model.predict(dff[['PTS']]), c='black')
plt.xlabel('PTS')
plt.ylabel('TOV')

Посмотрим на величину регрессионного коэффициента

In [ ]:
model.coef_

Ее положительность говорит нам, что с ростом числа заработанных очков растет число потерь

Теперь взглянем на корреляцию

In [ ]:
from scipy.stats import pearsonr

pearsonr(dff['PTS'], dff['TOV'])[0]

In [ ]:
from sklearn.metrics import r2_score

x = (dff['TOV'] - dff['TOV'].mean()) / dff['TOV'].std()
y = (dff['PTS'] - dff['PTS'].mean()) / dff['PTS'].std()
r2_score(x, y)

Корреляция, близкая к 1 говорит, что данные линейно зависимы. Ну и высокое значение коэффициента детерминации говорит, что `TOV` достаточно хорошо объясняется `PTS` (регрессия по х учитывает 80% дисперсии у)

In [ ]:
dff['prediction'] = model.predict(dff[['PTS']])[:,0]

dff[['Player','TOV','prediction']].head(4)

По четырем ответам видно, что линейная модель достаточно хорошо выявила зависимость для `TOV` из `PTS`.

 Теперь посмотрим на среднюю относительную ошибку

In [ ]:
((dff['TOV'] - dff['prediction']) / dff['TOV']).abs().mean()

Ошибка получилась значительной. Предположу, что это из-за выбросов сверху.